# PyConAr 2018 - Taller API Georef

Para leer la guía de uso de la API, hacer click [acá](https://apis.datos.gob.ar/georef).

Para leer la referencia completa de la API, hacer click [acá](https://datosgobar.github.io/georef-ar-api/open-api/).

Requerimientos del taller:
 - Python 3
 - jupyter
 - pandas
 - requests

In [1]:
import json
import os
import pandas as pd
import requests

## Ejemplos rápidos

A contiuación, se muestran algunos ejemplos rápidos de uso de la API, utilizando `requests`:

In [2]:
def georef_response(url):
    # Realizar una consulta GET a la API y devolver la respuesta JSON
    resp = requests.get('https://apis.datos.gob.ar/georef/api{}'.format(url))
    return resp.json()

georef_response('/provincias?nombre=pampa')

{'cantidad': 1,
 'inicio': 0,
 'provincias': [{'centroide': {'lat': -37.131554, 'lon': -65.446655},
   'id': '42',
   'nombre': 'La Pampa'}],
 'total': 1}

In [3]:
georef_response('/localidades?provincia=san juan&max=1')

{'cantidad': 1,
 'inicio': 0,
 'localidades': [{'centroide': {'lat': -31.416533, 'lon': -68.531614},
   'departamento': {'id': '70007', 'nombre': 'Albardón'},
   'id': '70007020001',
   'municipio': {'id': '700007', 'nombre': 'Albardón'},
   'nombre': 'CAMPO AFUERA',
   'provincia': {'id': '70', 'nombre': 'San Juan'},
   'tipo': 'Entidad (E)'}],
 'total': 99}

In [4]:
georef_response('/direcciones?direccion=santa fe 1000&departamento=quilmes&max=1')

{'cantidad': 1,
 'direcciones': [{'altura': 1000,
   'departamento': {'id': '06658', 'nombre': 'Quilmes'},
   'id': '0665801004045',
   'nombre': 'SANTA FE',
   'nomenclatura': 'SANTA FE 1000, Quilmes, Buenos Aires',
   'provincia': {'id': '06', 'nombre': 'Buenos Aires'},
   'tipo': 'CALLE',
   'ubicacion': {'lat': None, 'lon': None}}],
 'inicio': 0,
 'total': 2}

In [5]:
georef_response('/ubicacion?lat=-27.2741&lon=-66.7529')

{'ubicacion': {'departamento': {'id': '10035', 'nombre': 'Belén'},
  'lat': -27.2741,
  'lon': -66.7529,
  'municipio': {'id': '100077', 'nombre': 'Hualfín'},
  'provincia': {'id': '10', 'nombre': 'Catamarca'}}}

## Lotes de datos
Para trabajar con archivos de datos CSV, es conveniente utilizar los recursos versión `POST` de la API, ya que los mismos permiten envíar varias consultas en una misma petición HTTP. Para lograr esto, se definen a continuación algunas funciones para trabajar con datos cargados en DataFrames.

### Funciones de utilidad

Se definen dos funciones de utilidad:

#### `load_csv`
La función `load_csv` carga un archivo CSV y toma una muestra aleatoria de 100 elementos (por defecto). El muestreado se realiza para poder agilizar el taller y no tener que manejar cantidades de datos demasiado grandes.

In [6]:
def load_csv(name, length=100, random_state=1):
    # Cargar archivo CSV de la carpeta 'datos', eliminando duplicados
    dataframe = pd.read_csv(os.path.join('datos', name)).drop_duplicates()
    
    # Tomar una muestra 'aleatoria' de N elementos
    dataframe = dataframe.sample(n=min(len(dataframe.index), length), random_state=random_state)
    
    # Asegurarse de que los índices sean secuenciales
    return dataframe.reset_index(drop=True)

#### `expand_geojson_column`
La función `expand_geojson_column` toma un DataFrame con una columna `geojson`, y devuelve un nuevo DataFrame con dos columnas: `longitud` y `latitud`, con los valores correspondientes de la columna original. El contenido de cada celda de la columna `geojson` debe ser un valor de tipo `Point`.

In [7]:
def expand_geojson_column(data):
    # Toma un DataFrame con una columna 'geojson' y retorna un DataFrame con 'lat' y 'lon'
    def geojson_lat_lon(value):
        geojson = json.loads(value)
        return geojson['coordinates']
    
    col = data[['geojson']].applymap(geojson_lat_lon)
    return col['geojson'].apply(pd.Series).rename(columns={0: 'longitud', 1: 'latitud'})

Como ejemplo, se construye un DataFrame `geojson_df`, y se lo une con el resultado de expandir su columna `geojson` a `longitud` y `latitud`:

In [8]:
geojson_df = pd.DataFrame([
    '{"type":"Point","coordinates":[-65.5497560941612,-22.3231238419683]}',
    '{"type":"Point","coordinates":[-66.1589398249571,-22.3505830769888]}'
], columns=['geojson'])
geojson_df

,geojson
0,"{""type"":""Point"",""coordinates"":[-65.54975609416..."
1,"{""type"":""Point"",""coordinates"":[-66.15893982495..."


In [9]:
geojson_df.join(expand_geojson_column(geojson_df))

,geojson,longitud,latitud
0,"{""type"":""Point"",""coordinates"":[-65.54975609416...",-65.549756,-22.323124
1,"{""type"":""Point"",""coordinates"":[-66.15893982495...",-66.158940,-22.350583


### Funciones de normalización

Se definen dos funciones de normalización de datos:

#### `georef_normalize_entities`
La función `georef_normalize_entities` toma un DataFrame e intenta normalizar sus datos utilizando la API. Para utilizar la función, es necesario especificar qué recurso (`entity`) se va a consultar. Los valores aceptados por este parámetro son: `provincias`, `departamentos`, `municipios`, `localidades`, `direcciones` y `calles`. Por el otro lado, el parámetro `field` debe contener un diccionario que especifica cuáles campos del DataFrame corresponden a cada parámetro propio de la API.

In [10]:
def georef_normalize_entities(data, entity, fields, params=None):
    if not params:
        params = {}
    queries = []

    for i in data.index:
        query = {
            'max': 1,
            'aplanar': True,
            **params
        }

        for key, value in fields.items():
            query[key] = data.loc[i, value]

        queries.append(query)

    body = {
        entity: queries
    }

    url = 'https://apis.datos.gob.ar/georef/api/{}'.format(entity)

    resp = requests.post(url, json=body)
    resp.raise_for_status()
    results = resp.json()['resultados']

    return pd.DataFrame(result[entity][0] if result[entity] else {} for result in results)

A modo de ejemplo, se construye un DataFrame con un listado de provincias:

In [11]:
df_example = pd.DataFrame([
    'CABA',
    'Ciudad de Buenos Aires',
    'Capital federal',
    'Stgo del Estero',
    'Tierra del Fuego',
    'Tucuman'
], columns=['prov'])
df_example

,prov
0,CABA
1,Ciudad de Buenos Aires
2,Capital federal
3,Stgo del Estero
4,Tierra del Fuego
5,Tucuman


Luego, se normaliza el DataFrame utilizando `georef_normalize_entities`, con entidad `provincias`:

In [12]:
df_example.join(georef_normalize_entities(df_example, 'provincias', {'nombre': 'prov'}))

,prov,centroide_lat,centroide_lon,id,nombre
0,CABA,-34.614493,-58.445856,02,Ciudad Autónoma de Buenos Aires
1,Ciudad de Buenos Aires,-34.614493,-58.445856,02,Ciudad Autónoma de Buenos Aires
2,Capital federal,-34.614493,-58.445856,02,Ciudad Autónoma de Buenos Aires
3,Stgo del Estero,-27.782412,-63.252387,86,Santiago del Estero
4,Tierra del Fuego,-82.521518,-50.742749,94,"Tierra del Fuego, Antártida e Islas del Atlánt..."
5,Tucuman,-26.947800,-65.364758,90,Tucumán


#### `georef_normalize_entities`
La función `georef_reverse_geocode` toma un DataFrame con coordenadas (latitud y longitud) y realiza una geocodificación inversa de los datos utilizando la API. El parámetro `fields` debe tomar el valor de un diccionario que indique cuáles columnas del DataFrame corresponden a los parámetros `lat` y `lon` de la API.

In [13]:
def georef_reverse_geocode(data, fields, params=None):
    if not params:
        params = {}
    queries = []

    for i in data.index:
        query = {
            'aplanar': True,
            **params
        }

        for key, value in fields.items():
            query[key] = data.loc[i, value]

        queries.append(query)

    body = {
        'ubicaciones': queries
    }

    resp = requests.post('https://apis.datos.gob.ar/georef/api/ubicacion', json=body)
    resp.raise_for_status()
    results = resp.json()['resultados']

    return pd.DataFrame(result['ubicacion'] for result in results)

A modo de ejemplo, se construye un DataFrame con un listado de coordenadas:

In [14]:
df_example2 = pd.DataFrame([
    (-66.222703465532,-22.0225667586346),
    (-66.3782703614848,-22.7015782843536),
    (-68.4427937929881,-29.005534118598)
], columns=['longitud', 'latitud'])
df_example2

,longitud,latitud
0,-66.222703,-22.022567
1,-66.378270,-22.701578
2,-68.442794,-29.005534


Luego, se realiza la geocodificación inversa de los datos con `georef_reverse_geocode`:

In [15]:
df_example2.join(georef_reverse_geocode(df_example2, {'lat': 'latitud', 'lon': 'longitud'}))

,longitud,latitud,departamento_id,departamento_nombre,lat,lon,municipio_id,municipio_nombre,provincia_id,provincia_nombre
0,-66.222703,-22.022567,38077,Santa Catalina,-22.022567,-66.222703,386147,Santa Catalina,38,Jujuy
1,-66.378270,-22.701578,38049,Rinconada,-22.701578,-66.378270,386070,Mina Pirquitas,38,Jujuy
2,-68.442794,-29.005534,46077,General Lamadrid,-29.005534,-68.442794,460077,General Lamadrid,46,La Rioja
